In [ ]:
import numpy as np
import pythonToC
import parsing_yacc 
from parsing_lex import MyLexer
import pythonTopython2
import time

In [ ]:
# Define test input
input_code = """
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

c = np.add(a, b)
d = np.subtract(a, b)
e = np.sum(a)
f = np.dot(a, b)
"""

In [ ]:
# Create an instance of MyLexer
lexer = MyLexer()

# Build the lexer
lexer.build()

# Test the lexer on the input code
lexer.test(input_code)

In [ ]:
def deploy_parser(input, mode): 
    output = ""

    for line in input.split("\n"):
        if line == "":
            continue
        # print("line: ", line)   
        result = parsing_yacc.main(line, mode)
        # print("result: ", result)
        # Check if the result is a string 
        if isinstance(result, str):
            output += result + "\n"

    return output

In [ ]:
# Generate C code from Python input using yacc
print("Generating C code...")
#measure time to genreate C code - 
start_time = time.time()

c_output = deploy_parser(input_code, "C")
print("C code generation complete.")

c_time  = time.time() - start_time
print("Time taken to generate C code:", c_time)
print("\n")

print("c_output:", c_output)

if c_output:
    # Set up files for conversion
    pythonToC.set_up("test")
    pythonTopython2.setup("test")

    # Execute the input code to define variables
    exec(input_code)

    # Generate C code from Python input using pythontoC functions
    for line in input_code.split('\n'):
        if '=' in line:
            variable, value = map(str.strip, line.split("="))
            if '[' in value:  # Check if value is an array
                array_length = len(eval(value))  # Evaluate the array and get its length
                pythonToC.set_arr(variable, value, array_length, "test")
            else:  # Single value
                pythonToC.set_arr(variable, value, 1, "test")



    # Generate Python code from Python input using pythontopython functions
    for line in input_code.split('\n'):
        if '=' in line:
            exec("pythonTopython2." + line.strip())
    p_time  = time.time() - start_time
    

    # Generate Python code from C input using pythontopython functions
    #for line in c_output.split('\n'):
        #if line.strip():
            #exec("pythonTopython2." + line.strip())

    print("Conversion complete.")
else:
    print("No C output generated.")


In [ ]:
# Generate Python code from Python input using yacc
print("Generating Python code...")
start_time = time.time()

python_output = deploy_parser(input_code, "Python")
print("Python code generation complete.")

p_time  = time.time() - start_time
print("Time taken to generate Python code:", p_time)
print("\n")

print("python_output:", python_output)

if python_output:
    pass 
